1. Ler todos os arquivos 
2. Calcular os indicadores 
3. Enviar e-mail para cada uma das 25 lojas
4. Enviar e-mail da diretoria : 
4.1 Ranking das lojas (dia e ano)
4.2 Pior e melhor loja do dia e pior e melhor loja do ano
5. Salvar backup

In [48]:
import pandas as pd
from pathlib import Path
import smtplib
import email.message
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# importando os dados 
df_emails = pd.read_excel(r'Bases de Dados\Emails - Teste.xlsx')
df_lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding = 'latin1',sep=';')
df_vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

display(df_emails)
display(df_lojas)
display(df_vendas)

,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,ianshop.equipe@gmail.com
1,Shopping Midway Mall,Alice,ianshop.equipe@gmail.com
2,Norte Shopping,Laura,ianshop.equipe@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,ianshop.equipe@gmail.com
4,Shopping União de Osasco,Valentina,ianshop.equipe@gmail.com
5,Shopping Center Interlagos,Sophia,ianshop.equipe@gmail.com
6,Rio Mar Recife,Isabella,ianshop.equipe@gmail.com
7,Salvador Shopping,Heloisa,ianshop.equipe@gmail.com
8,Rio Mar Shopping Fortaleza,Luiza,ianshop.equipe@gmail.com
9,Shopping Center Leste Aricanduva,Julia,ianshop.equipe@gmail.com


,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping
3,4,Shopping Iguatemi Fortaleza
4,5,Shopping União de Osasco
5,6,Shopping Center Interlagos
6,7,Rio Mar Recife
7,8,Salvador Shopping
8,9,Rio Mar Shopping Fortaleza
9,10,Shopping Center Leste Aricanduva


,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71
...,...,...,...,...,...,...,...
100994,69996,2019-12-26,17,Short Listrado,2,102,204
100995,69996,2019-12-26,17,Mochila,4,270,1080
100996,69996,2019-12-26,17,Pulseira Estampa,1,87,87
100997,69997,2019-12-26,11,Camisa Listrado,1,108,108


ID Loja, Data, Faturamento dia
ID Loja, Data, Faturamento anual  

In [49]:
# Data mais recente nos dados 
dt_mais_recente = df_vendas['Data'].sort_values(ascending=False).iloc[0]
ano_corrente = dt_mais_recente.year

# df_faturamento_dia é um dataframe com faturamento na data mais recente presente nos dados
df_faturamento_dia = df_vendas[['Data', 'ID Loja','Valor Final']].groupby(['Data','ID Loja'],as_index=False).sum()
df_faturamento_dia = df_faturamento_dia.merge(df_lojas,on='ID Loja')
ordem = ['Data','ID Loja', 'Loja','Valor Final']
df_faturamento_dia = df_faturamento_dia[ordem]
df_faturamento_dia = df_faturamento_dia.rename(columns={'Valor Final': 'Faturamento'})
df_faturamento_dia = df_faturamento_dia[df_faturamento_dia['Data'] == dt_mais_recente]
display(df_faturamento_dia)

# df_faturamento_ano é um dataframe com faturamento do ano corrente 
df_vendas['Ano'] = df_vendas['Data'].dt.year
df_faturamento_ano = df_vendas.copy()
df_faturamento_ano = df_faturamento_ano[['Ano', 'ID Loja','Valor Final']].groupby(['Ano','ID Loja'],as_index=False).sum()
df_faturamento_ano = df_faturamento_ano.merge(df_lojas,on='ID Loja')
ordem = ['Ano','ID Loja', 'Loja','Valor Final']
df_faturamento_ano = df_faturamento_ano[ordem]
df_faturamento_ano =df_faturamento_ano.rename(columns={'Valor Final': 'Faturamento'})
df_faturamento_ano = df_faturamento_ano[df_faturamento_ano['Ano'] == ano_corrente]
display(df_faturamento_ano)

,Data,ID Loja,Loja,Faturamento
8927,2019-12-26,2,Shopping Midway Mall,170
8928,2019-12-26,3,Norte Shopping,1259
8929,2019-12-26,5,Shopping União de Osasco,1207
8930,2019-12-26,6,Shopping Center Interlagos,1582
8931,2019-12-26,8,Salvador Shopping,3950
8932,2019-12-26,9,Rio Mar Shopping Fortaleza,368
8933,2019-12-26,10,Shopping Center Leste Aricanduva,485
8934,2019-12-26,11,Ribeirão Shopping,650
8935,2019-12-26,14,Bourbon Shopping SP,676
8936,2019-12-26,15,Palladium Shopping Curitiba,381


,Ano,ID Loja,Loja,Faturamento
0,2019,1,Iguatemi Esplanada,1699681
1,2019,2,Shopping Midway Mall,1590441
2,2019,3,Norte Shopping,1711968
3,2019,4,Shopping Iguatemi Fortaleza,1674824
4,2019,5,Shopping União de Osasco,1663770
5,2019,6,Shopping Center Interlagos,1640486
6,2019,7,Rio Mar Recife,1722766
7,2019,8,Salvador Shopping,1646739
8,2019,9,Rio Mar Shopping Fortaleza,1698430
9,2019,10,Shopping Center Leste Aricanduva,1682870


In [50]:
# df_produtos_dia é um dataframe que contém a quantidade de produtos diferentes vendidos no dia mais recente 
df_produtos_dia = df_vendas[['Data', 'ID Loja','Produto']].groupby(['Data','ID Loja'],as_index=False).nunique()
df_produtos_dia = df_produtos_dia.merge(df_lojas,on='ID Loja')
ordem = ['Data','ID Loja', 'Loja','Produto']
df_produtos_dia = df_produtos_dia[ordem]
df_produtos_dia =df_produtos_dia.rename(columns={'Produto': 'Produtos diferentes vendidos'})
df_produtos_dia = df_produtos_dia[df_produtos_dia['Data'] == dt_mais_recente]
display(df_produtos_dia)
# df_produtos_ano é um dataframe que contém a quantidade de produtos diferentes vendidos no ano corrente
df_produtos_ano = df_vendas[['Ano','ID Loja','Produto']].groupby(['Ano','ID Loja'],as_index=False).nunique()
df_produtos_ano = df_produtos_ano.merge(df_lojas,on='ID Loja')
ordem = ['Ano','ID Loja', 'Loja','Produto']
df_produtos_ano = df_produtos_ano[ordem]
df_produtos_ano =df_produtos_ano.rename(columns={'Produto': 'Produtos diferentes vendidos'})
df_produtos_ano = df_produtos_ano[df_produtos_ano['Ano'] == ano_corrente]
display(df_produtos_ano)

,Data,ID Loja,Loja,Produtos diferentes vendidos
8927,2019-12-26,2,Shopping Midway Mall,1
8928,2019-12-26,3,Norte Shopping,2
8929,2019-12-26,5,Shopping União de Osasco,3
8930,2019-12-26,6,Shopping Center Interlagos,2
8931,2019-12-26,8,Salvador Shopping,2
8932,2019-12-26,9,Rio Mar Shopping Fortaleza,1
8933,2019-12-26,10,Shopping Center Leste Aricanduva,2
8934,2019-12-26,11,Ribeirão Shopping,3
8935,2019-12-26,14,Bourbon Shopping SP,1
8936,2019-12-26,15,Palladium Shopping Curitiba,2


,Ano,ID Loja,Loja,Produtos diferentes vendidos
0,2019,1,Iguatemi Esplanada,120
1,2019,2,Shopping Midway Mall,120
2,2019,3,Norte Shopping,120
3,2019,4,Shopping Iguatemi Fortaleza,120
4,2019,5,Shopping União de Osasco,120
5,2019,6,Shopping Center Interlagos,120
6,2019,7,Rio Mar Recife,120
7,2019,8,Salvador Shopping,120
8,2019,9,Rio Mar Shopping Fortaleza,120
9,2019,10,Shopping Center Leste Aricanduva,120


In [51]:
df_ticket_medio = df_vendas[['Data','ID Loja','Código Venda','Valor Final']].groupby(['Data','ID Loja','Código Venda'],as_index=False).sum()
df_ticket_medio = df_ticket_medio[['Data','ID Loja','Valor Final']].groupby(['Data','ID Loja'],as_index=False).mean()
df_ticket_medio = df_ticket_medio[df_ticket_medio['Data']== dt_mais_recente] 
df_ticket_medio = df_lojas.merge(df_ticket_medio,on=['ID Loja'])
df_ticket_medio = df_ticket_medio.rename(columns={'Valor Final': 'Ticket_medio'})
display(df_ticket_medio)

# df_ticket_medio_anual é um dataframe que contém o ticket médio por venda do ano corrente
df_ticket_medio_anual = df_vendas[['Data','ID Loja','Código Venda','Valor Final']].groupby(['Data','ID Loja','Código Venda'],as_index=False).sum()
df_ticket_medio_anual['Ano'] = df_ticket_medio_anual['Data'].dt.year
df_ticket_medio_anual = df_ticket_medio_anual[['ID Loja','Ano','Valor Final']].groupby(['Ano','ID Loja'],as_index=False).mean()
df_ticket_medio_anual =df_lojas.merge(df_ticket_medio_anual,on=['ID Loja'])
df_ticket_medio_anual = df_ticket_medio_anual.rename(columns={'Valor Final': 'Ticket_medio'})
display(df_ticket_medio_anual)

,ID Loja,Loja,Data,Ticket_medio
0,2,Shopping Midway Mall,2019-12-26,170.000000
1,3,Norte Shopping,2019-12-26,1259.000000
2,5,Shopping União de Osasco,2019-12-26,1207.000000
3,6,Shopping Center Interlagos,2019-12-26,1582.000000
4,8,Salvador Shopping,2019-12-26,3950.000000
5,9,Rio Mar Shopping Fortaleza,2019-12-26,368.000000
6,10,Shopping Center Leste Aricanduva,2019-12-26,485.000000
7,11,Ribeirão Shopping,2019-12-26,325.000000
8,14,Bourbon Shopping SP,2019-12-26,676.000000
9,15,Palladium Shopping Curitiba,2019-12-26,190.500000


,ID Loja,Loja,Ano,Ticket_medio
0,1,Iguatemi Esplanada,2019,794.986436
1,2,Shopping Midway Mall,2019,774.314021
2,3,Norte Shopping,2019,783.868132
3,4,Shopping Iguatemi Fortaleza,2019,772.876788
4,5,Shopping União de Osasco,2019,793.783397
5,6,Shopping Center Interlagos,2019,771.630292
6,7,Rio Mar Recife,2019,789.535289
7,8,Salvador Shopping,2019,753.656293
8,9,Rio Mar Shopping Fortaleza,2019,773.067820
9,10,Shopping Center Leste Aricanduva,2019,768.433790


In [52]:
# Função para formatar número em moeda 
def formatar (valor):
    valor_formatado = "{:,.0f}".format(valor)
    return valor_formatado

# Ranking do dia 
# Loja, Faturamento

df_ranking_dia = df_faturamento_dia[['Loja','Faturamento']].sort_values(by='Faturamento',ascending=False)
df_ranking_dia['Faturamento'] = df_ranking_dia['Faturamento'].apply(formatar)
display(df_ranking_dia)

# Ranking do ano 
df_ranking_ano = df_faturamento_ano[['Loja','Faturamento']].sort_values(by='Faturamento',ascending=False)
df_ranking_ano['Faturamento'] = df_ranking_ano['Faturamento'].apply(formatar)
display(df_ranking_ano)
# melhor do dia 
melhor_loja_dia = df_ranking_dia.iloc[0]
print("Melhor loja dia:",melhor_loja_dia)
# pior do dia 
pior_loja_dia = df_ranking_dia.iloc[-1]
print("Pior loja dia:",pior_loja_dia)

# melhor do ano
melhor_loja_ano = df_ranking_ano.iloc[0]
print("Melhor loja do ano",melhor_loja_ano)
# pior do ano
pior_loja_ano = df_ranking_ano.iloc[-1]
print("Pior loja do ano",pior_loja_ano)

#display(df_ranking_dia)
#display(df_ranking_ano)

,Loja,Faturamento
8931,Salvador Shopping,"3,950"
8942,Novo Shopping Ribeirão Preto,"3,400"
8937,Center Shopping Uberlândia,"2,651"
8940,Shopping Eldorado,"2,391"
8930,Shopping Center Interlagos,"1,582"
8938,Shopping Recife,"1,366"
8928,Norte Shopping,"1,259"
8929,Shopping União de Osasco,"1,207"
8939,Shopping Vila Velha,937
8935,Bourbon Shopping SP,676


,Loja,Faturamento
23,Iguatemi Campinas,"1,762,419"
18,Shopping Vila Velha,"1,731,167"
13,Bourbon Shopping SP,"1,726,110"
6,Rio Mar Recife,"1,722,766"
19,Shopping SP Market,"1,721,763"
14,Palladium Shopping Curitiba,"1,721,120"
2,Norte Shopping,"1,711,968"
10,Ribeirão Shopping,"1,707,122"
0,Iguatemi Esplanada,"1,699,681"
8,Rio Mar Shopping Fortaleza,"1,698,430"


Melhor loja dia: Loja           Salvador Shopping
Faturamento                3,950
Name: 8931, dtype: object
Pior loja dia: Loja           Shopping Ibirapuera
Faturamento                    118
Name: 8941, dtype: object
Melhor loja do ano Loja           Iguatemi Campinas
Faturamento            1,762,419
Name: 23, dtype: object
Pior loja do ano Loja           Shopping Morumbi
Faturamento           1,586,444
Name: 11, dtype: object


- As planilhas de cada loja devem ser salvas dentro da pasta da loja com a data da planilha, a fim de criar um histórico de backup


In [53]:
def semaforo_indicador(superou_meta):
    if superou_meta :
        url = 'https://images.emojiterra.com/google/noto-emoji/unicode-16.0/color/512px/1f7e2.png'
    else: 
        url = ('https://images.emojiterra.com/google/noto-emoji/unicode-16.0/color/1024px/1f534.png')

    return (f'<img src="{url}" width ="20">')

def enviar_email_gerente (nome_loja, nome_gerente,email, caminho,data,df_ind_dia,df_ind_ano):
    msg = MIMEMultipart()
    msg['Subject'] = f"OnePage do dia {data} - Loja {nome_loja}"
    msg['From'] = "lucasalvesmartins36@gmail.com"
    msg['To'] = email
    #msg['Cc'] = "lucasalvesmartins36+copia@gmail.com"
    #msg['Bcc'] = "ianshop.equipe+copia@gmail.com"
    
    # Estilo CSS inline para centralizar
    style_tabela = 'style="text-align: center; border-collapse: collapse;"'

    # Converte os DataFrames em HTML e aplica centralização
    df_ind_dia_html = df_ind_dia.to_html(index=False, escape=False, border=1)
    df_ind_dia_html = df_ind_dia_html.replace('<table border="1" class="dataframe">', f'<table {style_tabela}>')
    df_ind_dia_html = df_ind_dia_html.replace('<th>', '<th style="text-align:center;">')
    df_ind_dia_html = df_ind_dia_html.replace('<td>', '<td style="text-align:center;">')

    df_ind_ano_html = df_ind_ano.to_html(index=False, escape=False, border=1)
    df_ind_ano_html = df_ind_ano_html.replace('<table border="1" class="dataframe">', f'<table {style_tabela}>')
    df_ind_ano_html = df_ind_ano_html.replace('<th>', '<th style="text-align:center;">')
    df_ind_ano_html = df_ind_ano_html.replace('<td>', '<td style="text-align:center;">')

    corpo_email = f"""<p> Bom dia,{nome_gerente} </p> 
    <p> O resultado de ontem da loja {nome_loja} foi : </p>
    <p>{df_ind_dia_html}</p>
    <p>{df_ind_ano_html}</p>
    <p> Segue em anexo a planilha com todos os dados para mais detalhes. </p>
    <p> Qualquer dúvida, estou à disposição </p>
    <p> Att, Lucas </p> """
     
    msg.attach(MIMEText(corpo_email,"html"))
    
    caminho = Path(caminho)

    nome_anexo = caminho.name.replace("Backup_", "") 

    # anexar arquivos
    with open(caminho,"rb") as arquivo:
        msg.attach(MIMEApplication(arquivo.read(), Name = nome_anexo)) # conteúdo do arquivo e nome do arquivo no email

    servidor = smtplib.SMTP("smtp.gmail.com",587)
    servidor.starttls()
    servidor.login(msg['From'],"emqk gfwj muwi rvzw")
    servidor.send_message(msg)
    servidor.quit()

def cria_tabela_indicador (id) :
    """ Retorna  uma tabela com Faturamento, Produtos diferentes vendidos no dia e Ticket médio para a Loja cujo identificador é id """
    l_valores = []
    
    # Meta de faturamento do dia, produtos diferentes e ticket médio
    l_meta = [1000,4,500]

    if not df_faturamento_dia[df_faturamento_dia['ID Loja']==id].empty:
        # Houve vendas na data mais recente para essa loja
        
        l_valores.append(df_faturamento_dia[df_faturamento_dia['ID Loja']==id]['Faturamento'].values[0])
        l_valores.append(df_produtos_dia[df_produtos_dia['ID Loja']==id]['Produtos diferentes vendidos'].values[0])
        l_valores.append(df_ticket_medio[df_ticket_medio['ID Loja']==id]['Ticket_medio'].values[0])
    else:
        # Não houve vendas na data mais recente para essa loja
        l_valores.append(0)
        l_valores.append(0)
        l_valores.append(0)

    df_indicadores_dia = pd.DataFrame({
        'Indicador' : ["Faturamento", "Diversidade de produtos","Ticket médio por venda"],
        'Valor dia' : l_valores, 
        'Meta dia' : l_meta
        
        })        
    
    l_valores_ano = []
    l_valores_ano.append(df_faturamento_ano[df_faturamento_ano['ID Loja']==id]['Faturamento'].values[0])
    l_valores_ano.append(df_produtos_ano[df_produtos_ano['ID Loja']==id]['Produtos diferentes vendidos'].values[0])
    l_valores_ano.append(df_ticket_medio_anual[df_ticket_medio_anual['ID Loja']==id]['Ticket_medio'].values[0])

    # Meta de faturamento do dia, produtos diferentes e ticket médio
    l_meta_ano = [1650000,120,500]

    df_indicadores_ano = pd.DataFrame({
        'Indicador' : ["Faturamento", "Diversidade de produtos","Ticket médio por venda"],
        'Valor ano' : l_valores_ano, 
        'Meta ano' : l_meta_ano
    })

    df_indicadores_dia['Semaforo'] = df_indicadores_dia['Valor dia']>df_indicadores_dia['Meta dia']
    df_indicadores_dia['Semaforo'] = df_indicadores_dia['Semaforo'].apply(semaforo_indicador)

    df_indicadores_ano['Semaforo'] = df_indicadores_ano['Valor ano']>df_indicadores_ano['Meta ano']
    df_indicadores_ano['Semaforo'] = df_indicadores_ano['Semaforo'].apply(semaforo_indicador)

    ## Aplica a formatação nas colunas numéricas
    for col in ['Valor dia','Meta dia']:
        df_indicadores_dia[col] = df_indicadores_dia[col].apply(formatar)

    for col in ['Valor ano','Meta ano']:
        df_indicadores_ano[col] = df_indicadores_ano[col].apply(formatar)

    return df_indicadores_dia,df_indicadores_ano



In [ ]:
# Cria o backup e envia e-mail para os gerentes

for id in df_lojas['ID Loja']:
    str_caminho = r'C:\Users\Usuario\Curso Python Hashtag\Exercício Desafio pandas - Importação e Tratamento de Dados\Projeto Automacao Indicadores\Backup Arquivos Lojas' + '\\' + str(id)
    caminho = Path(str_caminho)
    caminho.mkdir(parents=True, exist_ok=True)
    data = dt_mais_recente.strftime('%Y-%m-%d')
    # Salvar a planilha de cada loja em sua respectiva pasta
    nome_arquivo = str_caminho+r'\Backup_Loja_'+str(id)+'_dt_'+ data + '.xlsx'
    df_vendas[df_vendas['ID Loja']==id].to_excel(nome_arquivo)
    
    #nome_loja = df_lojas.loc[df_lojas['ID Loja']==id,'Loja'].values[0]
    nome_loja = df_lojas[df_lojas['ID Loja']==id]['Loja'].values[0]
    nome_gerente = df_emails[df_emails['Loja']==nome_loja]['Gerente'].values[0]
    email_gerente = df_emails[df_emails['Loja']== nome_loja]['E-mail'].values[0]
    df_ind_dia,df_ind_ano = cria_tabela_indicador(id) 
    data_titulo_email = dt_mais_recente.strftime('%d-%m-%Y')
    enviar_email_gerente(nome_loja,nome_gerente,email_gerente,nome_arquivo,data_titulo_email,df_ind_dia,df_ind_ano)

In [55]:
# Envia e-mail para a diretoria 

msg = MIMEMultipart()
msg['Subject'] = f"OnePage do dia {data}"
msg['From'] = "lucasalvesmartins36@gmail.com"
msg['To'] = df_emails.loc[df_emails['Loja']=='Diretoria','E-mail'].values[0]
#msg['Cc'] = "lucasalvesmartins36+copia@gmail.com"
#msg['Bcc'] = "ianshop.equipe+copia@gmail.com"

df_ranking_dia_html = df_ranking_dia.to_html(index=False)
df_ranking_ano_html = df_ranking_ano.to_html(index=False)
melhor_loja_dia_txt = melhor_loja_dia.to_string().replace('\n', '<br>')
pior_loja_dia_txt = pior_loja_dia.to_string().replace('\n', '<br>')
melhor_loja_ano_txt = melhor_loja_ano.to_string().replace('\n', '<br>')
pior_loja_ano_txt = pior_loja_ano.to_string().replace('\n', '<br>')

corpo_email = f"""<p> Bom dia,</p> 
<p> Ranking de faturamento do dia : </p>
<p>{df_ranking_dia_html}</p>
<p> Ranking de faturamento do ano : </p>
<p>{df_ranking_ano_html}</p>
<p> Melhor loja do dia: {melhor_loja_dia_txt}  </p>
<p> Pior loja do dia: {pior_loja_dia_txt}  </p>
<p> Melhor loja do ano: {melhor_loja_ano_txt}  </p>
<p> Pior loja do ano: {pior_loja_ano_txt}  </p>

<p> Qualquer dúvida, estou à disposição </p>
<p> Att, Lucas </p> """
    
msg.attach(MIMEText(corpo_email,"html"))

caminho = Path(caminho)

# Falta o anexo
#nome_anexo = caminho.name.replace("Backup_", "") 

# anexar arquivos
#with open(caminho,"rb") as arquivo:
#    msg.attach(MIMEApplication(arquivo.read(), Name = nome_anexo)) # conteúdo do arquivo e nome do arquivo no email

servidor = smtplib.SMTP("smtp.gmail.com",587)
servidor.starttls()
servidor.login(msg['From'],"emqk gfwj muwi rvzw")
servidor.send_message(msg)
servidor.quit()

(221,
 b'2.0.0 closing connection d9443c01a7336-23fc5aac459sm54372995ad.105 - gsmtp')